In [31]:
from PyPDF2 import PdfReader
from langchain.text_splitter import CharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain_mistralai import ChatMistralAI 
import psycopg2
from sentence_transformers import SentenceTransformer
from langchain.vectorstores import PGVector

In [3]:
def get_pdf_content(documents):
    # Ensure documents is always treated as a list
    if isinstance(documents, str):
        documents = [documents]

    raw_text = ""

    for document in documents:
        try:
            pdf_reader = PdfReader(document)
            for page in pdf_reader.pages:
                text = page.extract_text()
                if text:  # Ensure non-empty text is appended
                    raw_text += text
        except FileNotFoundError:
            print(f"File not found: {document}")
        except Exception as e:
            print(f"An error occurred while processing {document}: {e}")

    return raw_text

In [4]:
def get_chunks(text):
    text_splitter = CharacterTextSplitter(
        separator="\n",
        chunk_size=1000,
        chunk_overlap=200,
        length_function=len
    )
    text_chunks = text_splitter.split_text(text)
    return text_chunks

In [ ]:
def get_embeddings(chunks):
    # Initialize HuggingFace embeddings using the specified model
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
    
    # Create a FAISS vector store from the text chunks and embeddings
    vector_storage = FAISS.from_texts(texts=chunks, embedding=embeddings)

    return vector_storage

In [29]:
def get_embeddings(chunks, filepaths):
    # Initialize HuggingFace embeddings using the specified model
    model = SentenceTransformer("sentence-transformers/all-mpnet-base-v2")
    
    # Generate embeddings for the text chunks
    embeddings = model.encode(chunks)

    # Connect to your PostgreSQL database
    conn = psycopg2.connect(
    host="localhost",  # e.g., "localhost"
    database="postgres",  # your database name
    user="postgres",  # your username
    password="Govindal@123"  # your password
    )
    cursor = conn.cursor()

    # Insert embeddings into the PostgreSQL table
    for embedding, textchunk, filepath in zip(embeddings, chunks, filepaths):
        # Insert data into the embeddings table
        cursor.execute("""
            INSERT INTO embeddings (embedding, textchunk, filepath)
            VALUES (%s, %s, %s);
        """, (embedding.tolist(), textchunk, filepath))  # Convert embedding to list

    # Commit the transaction and close the connection
    conn.commit()
    cursor.close()
    conn.close()

    print("Embeddings inserted successfully.")

In [7]:
def start_conversation(vector_embeddings, query):
    # Initialize Mistral AI's LLM (hypothetical)
    llm = ChatMistralAI(api_key="nR3fP67vOHHPRRK8VD63rQevrewxhjOo")
    
    # Set up memory for maintaining conversation history
    memory = ConversationBufferMemory(
        memory_key='chat_history',
        return_messages=True
    )
    
    # Create a conversational retrieval chain
    conversation = ConversationalRetrievalChain.from_llm(
        llm=llm,
        retriever=vector_embeddings.as_retriever(),
        memory=memory
    )
    
    # Generate a response for the provided query
    response = conversation.run(query)

    return response

In [10]:
file = '../data/trump_tariff.pdf'
text = get_pdf_content(file)

In [11]:
chunks = get_chunks(text)

In [32]:
print(chunks[1])

with the US, and this poli cy shift has rattled global markets , Reuters reports.  
 
Top three trading partners to the US - Mexico, Canada and China. Image: US Census 
Bureau/Reuters  
The Canadian dollar hit a 20 -year low, while the Me xican peso remains volatile, and the euro 
faces pressure amid fears the European Union is next. However, despite the tariffs, China’s 
yuan has remained stable.  
 
US futures and foreign currencies fluctuated following tarif f announcements. Image: 
Reuters/LSEG  
The auto sector has also warned of major profit losses, globally but particularly in the US, 
with analysts cautioning that ongoing uncertainty could slow economic growth and fuel 
inflation worldwide.  
US job growth slowed in January , impacted by weather and wildfires, while economists say a 
4.0% unemployment rate may delay Federal Reserve rate cuts  until June.  
Discover  
How is the World Economic Forum improving the global financial system?


In [14]:
conn = psycopg2.connect(
    host="localhost",  # e.g., "localhost"
    database="postgres",  # your database name
    user="postgres",  # your username
    password="Govindal@123"  # your password
)
cursor = conn.cursor()

In [30]:
get_embeddings(chunks,file)

Embeddings inserted successfully.
